# Momentos Invariantes de Hu

In [30]:
# imports
import cv2
from pathlib import Path
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [31]:
def hu_moments(filename, quadrants=(1,1)):
    #open img
    image = cv2.imread(str(filename), cv2.IMREAD_GRAYSCALE)

    # Tamanho dos quadrantes
    width, height = image.shape[0], image.shape[1]
    part_width = width // quadrants[0]
    part_height = height // quadrants[1]
    
    # Calculate
    huMoments = list()
    for i in range(quadrants[0]):
        for j in range(quadrants[1]):
            #coordenadas da parte da imagem
            left = i * part_width
            upper = j * part_height
            right = left + part_width
            lower = upper + part_height

            # Retorna o quadrante da imagem
            quad = image[left:right, upper:lower]

            # Calculate Moments 
            moments = cv2.moments(quad)
            # Calculate Hu Moments
            huMoments.extend(cv2.HuMoments(moments).flatten())
    return huMoments

def get_files(caminho_p):
    file_dict = {}
    for pasta in Path(caminho_p).iterdir():
      if pasta.is_dir():
        file_dict[pasta.name] = []
        
        for file in Path(pasta).iterdir():
            if file.is_file() and (file.name.endswith(".png") or file.name.endswith(".jpg")):
                file_dict[pasta.name].append(str(file))
    return file_dict

### Create X, y

In [32]:
database = get_files('./fourShapes/')

X, y = list(), list()

for classe in database:
    for item in database[classe]:
        X.append(hu_moments(item, (1,1)))
        y.append(classe)

X, y = np.array(X), np.array(y)
print(X.shape, y.shape)

(4000, 7) (4000,)


### k-NN

In [33]:
# normalize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# separa em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

print('Conjunto de treino (X, y):', X_train.shape, y_train.shape)
print('Conjunto de teste (X, y):', X_test.shape, y_test.shape)

Conjunto de treino (X, y): (2400, 7) (2400,)
Conjunto de teste (X, y): (1600, 7) (1600,)


In [34]:
#knn
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

# accuracy_score(y_test, y_pred)
# confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred, labels=knn.classes_))

              precision    recall  f1-score   support

      circle       0.83      0.94      0.88       386
      square       0.93      0.81      0.87       403
        star       1.00      1.00      1.00       399
    triangle       1.00      1.00      1.00       412

    accuracy                           0.94      1600
   macro avg       0.94      0.94      0.94      1600
weighted avg       0.94      0.94      0.94      1600

